In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import tensorflow as tf
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, ReLU, GlobalAveragePooling2D, Input

In [6]:
class_names = ['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings',
               'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse',
               'harbor', 'intersection', 'mediumresidential', 'mobilehomepark',
               'overpass', 'parkinglot', 'river', 'runway', 'sparseresidential',
               'storagetanks', 'tenniscourt']

In [7]:
# Instantiate the ResNet50 model
conv_base =tf.keras.applications.resnet.ResNet50(input_shape=(256, 256, 3),
                          include_top=False,
                          weights='imagenet')

# freeze the layers
for layer in conv_base.layers:
  layer.trainable = False

names = []
for layer in conv_base.layers:
  names.append(layer.name)

names[-1] # getting the name of the last conv layer

last_layer = conv_base.get_layer('conv5_block3_out')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

inputs = Input(shape = (256, 256, 3))
x = conv_base(inputs)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a final softmax layer for classification
output = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs, output)

last layer output shape:  (None, 8, 8, 2048)


In [9]:
os.chdir("drive/MyDrive/Challenge_dataset/")

In [10]:
model.load_weights('./checkpoints/my_checkpoint')

In [11]:
with open("test.npy", "rb") as f:
  X_val = np.load(f)
  y_val = np.load(f)


In [12]:
X_val = X_val / 255.0

In [13]:
classif_prob = model.predict(X_val)
pred_classes_argmax = np.argmax(classif_prob,axis=-1)

In [14]:
predicted_cls = pred_classes_argmax[0]
print("Predicted class:", predicted_cls)

Predicted class: 0


In [16]:
from tensorflow.keras.optimizers import RMSprop

# compile the model
model.compile(optimizer=RMSprop(lr=0.001),
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [17]:
values = model.evaluate(X_val, y_val)
print("{}:{},{}:{}.".format(model.metrics_names[0],values[0],model.metrics_names[1],values[1]))

9/9 [==============================] - 59s 6s/step - loss: 3.2318 - accuracy: 0.4167
loss:3.2318239212036133,accuracy:0.4166666567325592.
